# The Battle of Neighborhoods - segmenting and clustering magical Kenya's sceneries

### Install necessary libraries

In [ ]:
!conda install -c conda-forge geopy --yes # to determine latitude and longitude
!conda install -c conda-forge folium=0.5.0 --yes # to render maps

### Import necessary libraries

In [2]:
import pandas as pd
from geopy.geocoders import Nominatim 
import folium
import numpy as np
import json # library to handle JSON files
import requests # library to handle requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

pd.set_option('display.max_rows', None)

### Read in Kenyan neighborhoods

In [82]:
df = pd.read_csv('kenya_neighborhoods.csv', header=0)
df.head()

,Neighborhood
0,Mombasa
1,Nairobi
2,Kisumu
3,Diani
4,Kilifi


### Let us see the size of the read data

In [83]:
df.shape

(88, 1)

### Are there any duplicate neighborhood names?

In [84]:
len(df['Neighborhood'].unique())

88

The unique neighborhoods are 88, same as the dataframe size, so all are unique

### Create dataframe to hold the neighborhood name, latitude and longitude data

In [94]:
column_names =['Neighborhood','Latitude','Longitude']
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Neighborhood,Latitude,Longitude


In [95]:
neighborhood_only_df = df['Neighborhood']
neighborhood_only_df.head()

0    Mombasa
1    Nairobi
2     Kisumu
3      Diani
4     Kilifi
Name: Neighborhood, dtype: object

### Define function to get neighborhood's latitude and longitude from Geopy

In [96]:
def Get_Coordinates(neighborhoods_list):
    global neighborhoods
    for neighborhood in neighborhoods_list:             
        try:
            #get latitude and longitude for neighborhood using Geopy
            address = neighborhood + ", KE"
            geolocator = Nominatim(user_agent="ke_explorer")
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
            print('The geograpical coordinates of {} are {}, {}.'.format(neighborhood, latitude, longitude))      
            
            neighborhoods = neighborhoods.append({'Neighborhood': neighborhood,
                                  'Latitude':latitude,
                                  'Longitude':longitude},ignore_index=True)      
        except:
            print('Coordinates for {} were not found'.format(neighborhood))
            neighborhoods = neighborhoods.append({'Neighborhood': neighborhood,
                                  'Latitude': np.nan,
                                  'Longitude': np.nan},ignore_index=True)
            
            

### Get the coordinates for the neighborhoods and store in a dataframe

In [ ]:
Get_Coordinates(neighborhood_only_df)

In [153]:
neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,Mombasa,-4.039015,39.648391
1,Nairobi,-1.283253,36.817245
2,Kisumu,-0.102911,34.754176
3,Diani,-4.290581,39.585806
4,Kilifi,-3.150739,39.675072


In [155]:
neighborhoods.shape

(78, 3)

### Draw map of Kenya and add markers for each town using folium

In [133]:
#get latitude and longitude for Kenya using Geopy

address = 'Kenya, KE'
geolocator = Nominatim(user_agent="ke_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Kenya are {}, {}.'.format(latitude, longitude))

map_kenya = folium.Map(location=[latitude, longitude], zoom_start=10)
#map_kenya

The geograpical coordinates of Kenya are 1.4419683, 38.4313975.


In [102]:
# add markers to the Kenya map for each neighborhood and draw the map

for lat, lng,neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kenya)  
    
map_kenya

### Set parameters for Foursquare

In [103]:
CLIENT_ID = 'ZLBZ2XERFFWYQLPGZACQ2GQMXEXTOAUGBTRF4AGH3NE21WEI' # your Foursquare ID
CLIENT_SECRET = 'SER2GAHGZG2GOFYTQISZC1JGVRMI45UCYD0U5TZMX0FBRSGY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
RADIUS = 5000 # define radius. The maximum supported radius is currently 100,000 meters

### Define a function to get nearby venues from Foursquare

In [104]:
def getNearbyVenues(neighborhoods_list, latitudes, longitudes):
    
    venues_list=[]
    for neighborhood, lat, lng in zip(neighborhoods_list, latitudes, longitudes):
        print(neighborhood)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            RADIUS, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Town Latitude', 
                  'Town Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get the venues around each neighborhood from Foursquare

In [105]:
kenya_venues = getNearbyVenues(neighborhoods_list=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Mombasa
Nairobi
Kisumu
Diani
Kilifi
Kwale
Lamu
Malindi
Masai Mara
Naivasha
Nakuru
Nanyuki
Narok
Naromoru
Tsavo
Wasini
Watamu
Aberdare
Amboseli
Arabuko Sokoke
Central Island
Chyulu Hills
Hell's Gate
Lake Nakuru
Meru
Mombasa Marine Park
Mount Elgon National Park
Mount Kenya National Park
Mount Longonot
Nairobi National Park
Ol Donyo Sabuk
Ruma
Saiwa Swamp
Sibiloi
Tsavo East
Tsavo West
Bisanadi
Boni
Buffalo Springs
Dodori
Kakamega Forest
Kisumu Impala
Lake Bogoria
Mwea
Samburu
Shimba Hills
Witu Forest
Kiunga
Tana River
Baringo
Bogoria
Chala
Elmenteita
Jipe
Kamnarok
Kenyatta
Magadi
Lake Naivasha
Turkana
Lake Victoria
Bamburi
Funzi Island
Mtwapa
Tiwi Beach
Watamu Beach
Che Shale
Kikambala Beach
Dunga Beach
Turtle Bay
Shanzu
Vipingo Beach
Gazi
Galu
Kinondo beach
Chale Island
Msambweni beach
Shimoni
Wasini Island beach


### Below is the size of the dataframe of the venues

In [106]:
kenya_venues.shape

(2848, 7)

### Were all towns returned?

In [107]:
print('Neighborhoods with returned venues are ', len(kenya_venues['Neighborhood'].unique()))

Neighborhoods with returned venues are  78


So, all the neighborhoods were returned with their venues

### Below is the dataframe with the venues returned by Foursquare

In [108]:
kenya_venues.head(2)

,Neighborhood,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mombasa,-4.039015,39.648391,Tamarind Dhow,-4.049301,39.681443,Seafood Restaurant
1,Mombasa,-4.039015,39.648391,Barka Restaurant,-4.061477,39.676374,African Restaurant


### Let us see the venues returned for one neighborhood

In [156]:
kenya_venues[kenya_venues['Neighborhood']=='Malindi'].reset_index(drop=True)

,Neighborhood,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malindi,-3.216599,40.116593,Karen Brixen Cafe Malindi,-3.209104,40.117532,Café
1,Malindi,-3.216599,40.116593,"Osteria del Chianti, Malindi",-3.219514,40.125946,Italian Restaurant
2,Malindi,-3.216599,40.116593,Malindi Marine National Park,-3.254740,40.131856,Beach
3,Malindi,-3.216599,40.116593,Simba Dishes Malindi,-3.217145,40.118823,African Restaurant
4,Malindi,-3.216599,40.116593,Medina Palms,-3.361686,40.003833,Beach
5,Malindi,-3.216599,40.116593,Stardust,-3.209317,40.117013,Nightclub
6,Malindi,-3.216599,40.116593,Pata Pata Beach Club,-3.197697,40.121690,Cocktail Bar
7,Malindi,-3.216599,40.116593,Nakumatt Malindi,-3.209003,40.116909,Shopping Mall
8,Malindi,-3.216599,40.116593,Come Back Disco Lounge Bar Watamu,-3.354508,40.019880,Nightclub
9,Malindi,-3.216599,40.116593,Malindi Town,-3.219689,40.117189,General Travel


### We see the venues that were returned per neighborhood

In [110]:
kenya_venues.groupby('Neighborhood').count()

,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aberdare,32,32,32,32,32,32
Amboseli,21,21,21,21,21,21
Arabuko Sokoke,37,37,37,37,37,37
Bamburi,52,52,52,52,52,52
Baringo,4,4,4,4,4,4
Bisanadi,5,5,5,5,5,5
Bogoria,5,5,5,5,5,5
Boni,100,100,100,100,100,100
Buffalo Springs,24,24,24,24,24,24


### How many unique categories can be curated from all the returned venues

In [111]:
print('There are {} uniques venue categories.'.format(len(kenya_venues['Venue Category'].unique())))

There are 204 uniques venue categories.


# We analyze each town

In [134]:
# one hot encoding
kenya_onehot = pd.get_dummies(kenya_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kenya_onehot['Neighborhood'] = kenya_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kenya_onehot.columns[-1]] + list(kenya_onehot.columns[:-1])
kenya_onehot = kenya_onehot[fixed_columns]

kenya_onehot.head()

,Zoo,ATM,African Restaurant,Airport,Airport Lounge,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [135]:
kenya_onehot.shape

(2848, 204)

### Group rows by neighborhood and take the mean of the frequency of occurrence of each venue category per neighborhood

In [136]:
kenya_grouped = kenya_onehot.groupby('Neighborhood').mean().reset_index()
kenya_grouped

,Neighborhood,Zoo,ATM,African Restaurant,Airport,Airport Lounge,American Restaurant,Arcade,Art Gallery,Art Museum,...,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Wine Shop,Women's Store
0,Aberdare,0.00,0.000000,0.093750,0.000000,0.00000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Amboseli,0.00,0.190476,0.000000,0.047619,0.00000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,Arabuko Sokoke,0.00,0.000000,0.027027,0.027027,0.00000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Bamburi,0.00,0.000000,0.038462,0.000000,0.00000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Baringo,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,Bisanadi,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,Bogoria,0.00,0.000000,0.200000,0.000000,0.00000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,Boni,0.01,0.000000,0.000000,0.000000,0.00000,0.000000,0.010000,0.00,0.00,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.02
8,Buffalo Springs,0.00,0.000000,0.000000,0.000000,0.00000,0.041667,0.000000,0.00,0.00,...,0.041667,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,Central Island,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### Here is the size of the grouped dataframe

In [157]:
kenya_grouped.shape

(78, 204)

### Let us see the 3 commonest/top venues per neighborhood

In [158]:
num_top_venues = 3

for neighborhood in kenya_grouped['Neighborhood']:
    print("----"+neighborhood+"----")
    temp = kenya_grouped[kenya_grouped['Neighborhood'] == neighborhood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aberdare----
       venue  freq
0      Hotel  0.19
1  BBQ Joint  0.12
2     Resort  0.09


----Amboseli----
           venue  freq
0          Hotel  0.19
1            ATM  0.19
2  National Park  0.14


----Arabuko Sokoke----
                venue  freq
0               Hotel  0.16
1              Resort  0.16
2  Italian Restaurant  0.11


----Bamburi----
                venue  freq
0              Resort  0.13
1  Seafood Restaurant  0.13
2               Beach  0.10


----Baringo----
             venue  freq
0           Resort  0.25
1  Nature Preserve  0.25
2      Bus Station  0.25


----Bisanadi----
                 venue  freq
0                 Park   0.2
1  Rental Car Location   0.2
2    Convenience Store   0.2


----Bogoria----
                venue  freq
0               Hotel   0.2
1  African Restaurant   0.2
2              Resort   0.2


----Boni----
                  venue  freq
0                 Beach  0.17
1  Caribbean Restaurant  0.09
2           Pizza Place  0.06


----Buffa

### Create a dataframe of top 10 venues per town

Define a function to sort the venues in descending order

In [180]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe with the top 10 venues for each town.

In [181]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = kenya_grouped['Neighborhood']

for ind in np.arange(kenya_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(kenya_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aberdare,Hotel,BBQ Joint,Resort,Café,African Restaurant,Lounge,Miscellaneous Shop,Moving Target,Nightclub,Stables
1,Amboseli,Hotel,ATM,National Park,Park,Scenic Lookout,Trail,Mountain,RV Park,Tourist Information Center,Resort
2,Arabuko Sokoke,Hotel,Resort,Beach,Italian Restaurant,Nightclub,Golf Course,General Travel,Cocktail Bar,Shopping Mall,Fast Food Restaurant
3,Bamburi,Seafood Restaurant,Resort,Beach,Hotel,Italian Restaurant,African Restaurant,Restaurant,Coffee Shop,Café,Golf Course
4,Baringo,Lake,Bus Station,Nature Preserve,Resort,Diner,Fast Food Restaurant,French Restaurant,Forest,Food Court,Food & Drink Shop


# Cluster the towns of Kenya using k-means algorithm

In [182]:
# set number of clusters to be 5
kclusters = 5

kenya_grouped_clustering = kenya_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kenya_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 1, 1, 4, 1, 1, 4, 1], dtype=int32)

### Create a new dataframe that includes the cluster and the top 10 venues for each town

In [183]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

kenya_merged = neighborhoods

# merge kenya_grouped with towns to add latitude/longitude for each town
kenya_merged = kenya_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')

kenya_merged.head() 

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mombasa,-4.039015,39.648391,1,Seafood Restaurant,Resort,Hotel,Beach,Golf Course,Italian Restaurant,African Restaurant,Restaurant,Coffee Shop,Café
1,Nairobi,-1.283253,36.817245,4,Hotel,Café,Lounge,Shopping Mall,Coffee Shop,African Restaurant,Bar,Restaurant,Italian Restaurant,Bistro
2,Kisumu,-0.102911,34.754176,4,Shopping Mall,Hotel,Lounge,Nightclub,Department Store,Diner,Bar,Bed & Breakfast,Farm,Fast Food Restaurant
3,Diani,-4.290581,39.585806,1,Seafood Restaurant,Resort,Hotel,Beach,Italian Restaurant,Café,African Restaurant,Restaurant,Coffee Shop,Beach Bar
4,Kilifi,-3.150739,39.675072,1,Resort,Hotel,Beach,Café,Nightclub,Gastropub,Airport,African Restaurant,Hotel Bar,Cocktail Bar


In [184]:
kenya_merged

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mombasa,-4.039015,39.648391,1,Seafood Restaurant,Resort,Hotel,Beach,Golf Course,Italian Restaurant,African Restaurant,Restaurant,Coffee Shop,Café
1,Nairobi,-1.283253,36.817245,4,Hotel,Café,Lounge,Shopping Mall,Coffee Shop,African Restaurant,Bar,Restaurant,Italian Restaurant,Bistro
2,Kisumu,-0.102911,34.754176,4,Shopping Mall,Hotel,Lounge,Nightclub,Department Store,Diner,Bar,Bed & Breakfast,Farm,Fast Food Restaurant
3,Diani,-4.290581,39.585806,1,Seafood Restaurant,Resort,Hotel,Beach,Italian Restaurant,Café,African Restaurant,Restaurant,Coffee Shop,Beach Bar
4,Kilifi,-3.150739,39.675072,1,Resort,Hotel,Beach,Café,Nightclub,Gastropub,Airport,African Restaurant,Hotel Bar,Cocktail Bar
5,Kwale,-4.183607,39.105096,4,ATM,Rental Car Location,Farm,Fried Chicken Joint,French Restaurant,Forest,Food Court,Food & Drink Shop,Food,Flea Market
6,Lamu,-2.284674,40.864033,1,Hotel,Airport,Boat or Ferry,Resort,Dessert Shop,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Department Store,Forest
7,Malindi,-3.216599,40.116593,1,Hotel,Beach,Resort,Italian Restaurant,Nightclub,African Restaurant,Café,Food,Shopping Mall,General Travel
8,Masai Mara,-1.285418,36.853290,4,Hotel,Café,Lounge,Shopping Mall,Coffee Shop,African Restaurant,Bar,Restaurant,Italian Restaurant,Bistro
9,Naivasha,-0.721480,36.438738,1,Resort,Hotel,Scenic Lookout,Café,BBQ Joint,Outdoors & Recreation,African Restaurant,Travel & Transport,Miscellaneous Shop,Breakfast Spot


### Visualize the resulting clusters

In [185]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kenya_merged['Latitude'], kenya_merged['Longitude'], kenya_merged['Neighborhood'], kenya_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Have a look at each of the clusters

Cluster 1

In [193]:
kenya_merged.loc[kenya_merged['Cluster Labels'] == 0, kenya_merged.columns[[0] + list(range(4, kenya_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
55,Tana River,Italian Restaurant,Women's Store,Farm,Fried Chicken Joint,French Restaurant,Forest,Food Court,Food & Drink Shop,Food,Flea Market


Cluster 2

In [194]:
kenya_merged.loc[kenya_merged['Cluster Labels'] == 1, kenya_merged.columns[[0] + list(range(4, kenya_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mombasa,Seafood Restaurant,Resort,Hotel,Beach,Golf Course,Italian Restaurant,African Restaurant,Restaurant,Coffee Shop,Café
3,Diani,Seafood Restaurant,Resort,Hotel,Beach,Italian Restaurant,Café,African Restaurant,Restaurant,Coffee Shop,Beach Bar
4,Kilifi,Resort,Hotel,Beach,Café,Nightclub,Gastropub,Airport,African Restaurant,Hotel Bar,Cocktail Bar
6,Lamu,Hotel,Airport,Boat or Ferry,Resort,Dessert Shop,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Department Store,Forest
7,Malindi,Hotel,Beach,Resort,Italian Restaurant,Nightclub,African Restaurant,Café,Food,Shopping Mall,General Travel
9,Naivasha,Resort,Hotel,Scenic Lookout,Café,BBQ Joint,Outdoors & Recreation,African Restaurant,Travel & Transport,Miscellaneous Shop,Breakfast Spot
10,Nakuru,Hotel,African Restaurant,Resort,Café,Shopping Mall,Coffee Shop,BBQ Joint,Nightclub,Bar,Sports Club
11,Nanyuki,Hotel,Resort,Hotel Bar,Miscellaneous Shop,Shopping Mall,French Restaurant,Coffee Shop,Campground,Outlet Mall,Park
12,Narok,Hotel,Resort,Campground,Airport,National Park,Coffee Shop,Nature Preserve,Forest,Convenience Store,Restaurant
13,Naromoru,Hotel,Resort,Shopping Mall,Coffee Shop,Café,Hotel Bar,Lounge,Nightclub,African Restaurant,American Restaurant


Cluster 3

In [195]:
kenya_merged.loc[kenya_merged['Cluster Labels'] == 2, kenya_merged.columns[[0] + list(range(4, kenya_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Samburu,Home Service,Women's Store,Event Space,French Restaurant,Forest,Food Court,Food & Drink Shop,Food,Flea Market,Fish Market


Cluster 4

In [196]:
kenya_merged.loc[kenya_merged['Cluster Labels'] == 3, kenya_merged.columns[[0] + list(range(4, kenya_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Kiunga,Pub,Women's Store,Event Space,French Restaurant,Forest,Food Court,Food & Drink Shop,Food,Flea Market,Fish Market


Cluster 5

In [197]:
kenya_merged.loc[kenya_merged['Cluster Labels'] == 4, kenya_merged.columns[[0] + list(range(4, kenya_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Nairobi,Hotel,Café,Lounge,Shopping Mall,Coffee Shop,African Restaurant,Bar,Restaurant,Italian Restaurant,Bistro
2,Kisumu,Shopping Mall,Hotel,Lounge,Nightclub,Department Store,Diner,Bar,Bed & Breakfast,Farm,Fast Food Restaurant
5,Kwale,ATM,Rental Car Location,Farm,Fried Chicken Joint,French Restaurant,Forest,Food Court,Food & Drink Shop,Food,Flea Market
8,Masai Mara,Hotel,Café,Lounge,Shopping Mall,Coffee Shop,African Restaurant,Bar,Restaurant,Italian Restaurant,Bistro
14,Tsavo,ATM,Hotel,Convenience Store,Food Court,Bus Station,Campground,Building,Scenic Lookout,Park,Train Station
18,Amboseli,Hotel,ATM,National Park,Park,Scenic Lookout,Trail,Mountain,RV Park,Tourist Information Center,Resort
21,Chyulu Hills,Park,Bus Station,Hotel,Train Station,History Museum,Food Court,Rest Area,Restaurant,Convenience Store,Campground
22,Hell's Gate,Brewery,Hotel,Coffee Shop,American Restaurant,Pizza Place,Restaurant,Italian Restaurant,Rest Area,Café,Sandwich Place
24,Lake Nakuru,Hotel,African Restaurant,Shopping Mall,Resort,Coffee Shop,Nightclub,Bar,BBQ Joint,Lounge,Sports Club
28,Mount Elgon National Park,Hotel,Garden,Food,Bus Station,Frozen Yogurt Shop,French Restaurant,Forest,Food Court,Food & Drink Shop,Flea Market


### Concluding on the clusters generated 

Cluster 1 consists of one neighborhood only whose top venues are an Italian restaurant and farm

Cluster 2 does consist of the beautiful beaches of Kenya and are found at the Indian Ocean coast

Cluster 3 and 4 each consist of one neighborhood only

Cluster 5 has many of the national parks where animals can be viewed

Kenya is a beautiful country consisting of beaches, nationals parks and all the other basic ammenities like hotels and shopping malls

# The End